In [9]:
#pip install PyPDF2

In [6]:
import os
import pandas as pd
from PyPDF2 import PdfReader
import re

In [7]:
def delete_specific_files(directory_path, patterns):
    file_count = 0

    for filename in os.listdir(directory_path):
        # Check if the filename contains any of the specified patterns
        if any(pattern in filename for pattern in patterns):
            file_path = os.path.join(directory_path, filename)
            os.remove(file_path)  # Delete the file
            file_count += 1
            print(f"Deleted file: {filename}")

    print(f"Total files deleted: {file_count}")

# Directory path
directory_path = 'C:\\Users\\eugen\\corpus_article_debt\\raw files\\raw docs FR'

# Patterns to search for in filenames
patterns = ["Attachment", "(2)", "(3)", "(4)"]

# Execute the function
delete_specific_files(directory_path, patterns)

Total files deleted: 0


In [8]:
# Function 1: Extract title, text, and length
def extract_title_text_length(text):
    # Extract Title
    title = text.split('\n')[0]
    body_index = text.find('Body')
    article_text = text[body_index + len('Body'):].strip()
    
    # Try to find the length by looking for any characters after "Length:"
    length_match = re.search(r'Length:\s?(.+)', text)
    
    if length_match:
        length_str = length_match.group(1).strip()  # Extract the entire string and remove leading/trailing spaces
        # Check if the string contains only digits
        if length_str.isdigit():
            length = int(length_str)  # Convert to integer if it contains only digits
        else:
            length = length_str  # Keep as string if it contains other characters
    else:
        length = None

    return title, length

In [9]:
# Function 2: Extract the date and convert it to the correct date
def extract_date(text):
    # Mapping of French days and months to English
    day_mapping = {
        'Lundi': 'Monday', 'Mardi': 'Tuesday', 'Mercredi': 'Wednesday', 
        'Jeudi': 'Thursday', 'Vendredi': 'Friday', 'Samedi': 'Saturday', 
        'Dimanche': 'Sunday'
    }
    month_mapping = {
        'Janvier': 'January', 'Février': 'February', 'Mars': 'March', 'Avril': 'April',
        'Mai': 'May', 'Juin': 'June', 'Juillet': 'July', 'Août': 'August',
        'Septembre': 'September', 'Octobre': 'October', 'Novembre': 'November', 'Décembre': 'December'
    }

    # Define date pattern in French
    date_pattern = r'(?:Lundi|Mardi|Mercredi|Jeudi|Vendredi|Samedi|Dimanche)?,?\s?\d{1,2} (Janvier|Février|Mars|Avril|Mai|Juin|Juillet|Août|Septembre|Octobre|Novembre|Décembre) \d{4}'
    
    date_match = re.search(date_pattern, text, re.IGNORECASE)
    date = None

    if date_match:
        date_str = date_match.group(0)
        # Replace French days and months with English
        for fr, en in day_mapping.items():
            date_str = re.sub(fr, en, date_str, flags=re.IGNORECASE)
        for fr, en in month_mapping.items():
            date_str = re.sub(fr, en, date_str, flags=re.IGNORECASE)

        # Attempt to parse the date string into a datetime object
        try:
            date = pd.to_datetime(date_str, errors='coerce', dayfirst=True)
            if date is not None and not pd.isnull(date):
                date = date.strftime('%Y-%m-%d')
            else:
                date = date_match.group(0)  # Fallback: Return the original extracted date string
        except ValueError:
            date = date_match.group(0)  # Fallback: Return the original extracted date string

    return date

In [10]:
# Function 3: Extract the newspaper name
def extract_newspaper_name_from_start(text, newspaper_list, lines_to_check=8):
    newspaper_name = None
    lines = text.split('\n')[:lines_to_check]  # Split the text into lines and take the first few lines
    for line in lines:
        for newspaper in newspaper_list:
            if newspaper.lower() in line.lower():
                newspaper_name = newspaper
                return newspaper_name
    return newspaper_name

# Newspaper list
newspaper_list = [
    'Le Monde',
    'Le Figaro',
    'Libération',
    'La Croix',
    'Le Parisien',
    "L'Humanité",
    'Les Échos',
    'La Tribune',
    'Le Canard Enchaîné',
    "L'Équipe",
    'Le Nouvel Observateur',
    'Le Point',
    "L'Express",
    'Mediapart',
    'Charlie Hebdo',
    'Ouest-France',
    'Le Temps',
    'Le Dauphiné Libéré',
    'La Voix du Nord',
    "L'Aisne Nouvelle",
    'Le Télégramme',
    'La Montagne',
    'La Provence',
    "Le Parisien - Aujourd'hui en France",
    'Courrier International'
]

In [11]:
#Function 4: Extract text correctly without additional "Load-Date" etc
def extract_text_from_pdf(pdf_path):
    text = ''
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() if page.extract_text() else ''
    return text

start_indicator = "Body"
end_indicator = "Load-Date"

def identify_text_body(text, start_indicator, end_indicator):
    start_index = text.find(start_indicator)
    if start_index != -1:
        start_index += len(start_indicator)  # Move start index to the end of the start_indicator
    else:
        start_index = 0  # Start from the beginning if start_indicator not found
    
    end_index = text.find(end_indicator, start_index) if end_indicator in text and start_index != -1 else len(text)
    
    if start_index != -1 and end_index != -1:
        return text[start_index:end_index].strip()
    else:
        return text[start_index:].strip()  # Return the text from start_indicator to the end if end_indicator is not found

In [12]:
# Function 5: Identify the language
def identify_language(folder_path):
    """
    Identifies the language based on markers in the folder path.

    Parameters:
    - folder_path (str): The path of the folder which likely contains language markers.

    Returns:
    - str: The identified language ('French', 'German', or 'Unknown').
    """
    if "FR" in folder_path:
        return 'French'
    elif "DE" in folder_path:
        return 'German'
    else:
        return 'Unknown'

In [20]:
def clean_text(text):
    """Clean text by removing or replacing null bytes and control characters.
    
    If text is None, return an empty string."""
    if text is None:
        return ''
    return ''.join(char for char in text if char.isprintable())

In [21]:
def compile_and_save_corpus(directory_path, output_xlsx_path):
    file_count = 0  # To keep track of how many files have been processed
    
    # Open the Excel writer outside the loop
    writer = pd.ExcelWriter(output_xlsx_path, engine='openpyxl')
    
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            file_path = os.path.join(directory_path, filename)
            print(f"Processing file: {filename}")  # Debugging statement
            
            text = extract_text_from_pdf(file_path)
            title, length = extract_title_text_length(text)
            article_text = identify_text_body(text, 'start_indicator', 'end_indicator')
            date = extract_date(text)
            newspaper_name = extract_newspaper_name_from_start(text, 'newspaper_list', lines_to_check=7)
            language = identify_language(directory_path)
            
            # Apply cleaning to the extracted text data
            title = clean_text(title)
            article_text = clean_text(article_text)
            newspaper_name = clean_text(newspaper_name)
            
            # Create a DataFrame for the current file's data
            corpus_df = pd.DataFrame([{
                'Title': title,
                'Text Body': article_text,
                'Length': length,
                'Date': date,
                'Newspaper Name': newspaper_name,
                'Language': language
            }])
            
            # Write the current DataFrame to Excel file, appending if it's not the first file
            if file_count == 0:
                corpus_df.to_excel(writer, index=False)
            else:
                corpus_df.to_excel(writer, index=False, header=None, startrow=file_count + 1)
            
            file_count += 1
            print(f"File processed: {filename}")  # Debugging statement
    
    # Save and close the Excel writer after all files have been processed
    writer.save()
    writer.close()
    print(f"Total files processed: {file_count}")  # Debugging statement
    print(f"Corpus saved to: {output_xlsx_path}")  # Debugging statement

In [ ]:
# Run the main function
# Run it on French documents
directory_path = 'C:\\Users\\eugen\\corpus_article_debt\\raw files\\raw docs FR'
output_xlsx_path = 'C:\\Users\\eugen\\corpus_article_debt\\corpus_article_debt_FR.xlsx'
compile_and_save_corpus(directory_path, output_xlsx_path)

Processing file: 2375 milliards d'euros_ faut-il avoir peur de l'abyssale dette fran_aise_.pdf
File processed: 2375 milliards d'euros_ faut-il avoir peur de l'abyssale dette fran_aise_.pdf
Processing file: 57,4 milliards de dollars _ L'appui de la Banque mondiale aux pays en d_veloppement.pdf
File processed: 57,4 milliards de dollars _ L'appui de la Banque mondiale aux pays en d_veloppement.pdf
Processing file: A 63 _ des questions sur l _largissement.pdf
File processed: A 63 _ des questions sur l _largissement.pdf
Processing file: A Bruxelles, une r_forme minime du Pacte de stabilit_ budg_taire.pdf
File processed: A Bruxelles, une r_forme minime du Pacte de stabilit_ budg_taire.pdf
Processing file: A Djibouti , Macron vante des partenariats _respectueux_.pdf
File processed: A Djibouti , Macron vante des partenariats _respectueux_.pdf
Processing file: A Paris, un sommet pour _viter le d_crochage _conomique de l'Afrique.pdf
File processed: A Paris, un sommet pour _viter le d_crochage _c

Exception ignored in: <function ZipFile.__del__ at 0x00000238F8E7C430>
Traceback (most recent call last):
  File "C:\Users\eugen\anaconda3\lib\zipfile.py", line 1816, in __del__
    self.close()
  File "C:\Users\eugen\anaconda3\lib\zipfile.py", line 1833, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


File processed: Alain Supiot_ _Des urnes au travail, nous assistons _ la s_cession des gens ordinaires_.pdf
Processing file: Alerte _ la hausse des taux.pdf
File processed: Alerte _ la hausse des taux.pdf
Processing file: Alexandre Farnoux _ _ En Gr_ce, il faudrait un plan Marshall des ruines _.pdf
File processed: Alexandre Farnoux _ _ En Gr_ce, il faudrait un plan Marshall des ruines _.pdf
Processing file: Allemagne _ _quation budg_taire complexe pour le nouveau gouvernement.pdf
File processed: Allemagne _ _quation budg_taire complexe pour le nouveau gouvernement.pdf
Processing file: Allemagne_ Scholz veut transformer l'_conomie comme jamais depuis 100 ans.pdf
File processed: Allemagne_ Scholz veut transformer l'_conomie comme jamais depuis 100 ans.pdf
Processing file: ANALYSE L'agenda europ_en encalmin_ par les rat_s du moteur allemand.pdf
File processed: ANALYSE L'agenda europ_en encalmin_ par les rat_s du moteur allemand.pdf
Processing file: ANALYSE Non, l'Allemagne n'est pas l'hom

File processed: Bruno Le Maire en Haute-Savoie jeudi pour sa rentr_e.pdf
Processing file: Bruno Le Maire et Olaf Scholz _  _ Pour la premi_re fois, l'Europe agit unie _.pdf
File processed: Bruno Le Maire et Olaf Scholz _  _ Pour la premi_re fois, l'Europe agit unie _.pdf
Processing file: Bruno Le Maire _ _ La France ne se r_duit pas _ son _tat _.pdf
File processed: Bruno Le Maire _ _ La France ne se r_duit pas _ son _tat _.pdf
Processing file: Bruno Le Maire _ _ La politique a besoin de r_sultats _.pdf
File processed: Bruno Le Maire _ _ La politique a besoin de r_sultats _.pdf
Processing file: Bruno Le Maire _ _ Le capitalisme est dans une impasse _.pdf
File processed: Bruno Le Maire _ _ Le capitalisme est dans une impasse _.pdf
Processing file: Bruxelles envisage la fin de la c_l_bre r_gle des 3% explication La Commission europ_enne a relanc_.pdf
File processed: Bruxelles envisage la fin de la c_l_bre r_gle des 3% explication La Commission europ_enne a relanc_.pdf
Processing file: Bru

File processed: Ce que le syst_me financier peut faire pour le climat.pdf
Processing file: Ce que nous enseignent les crises du pass_.pdf
File processed: Ce que nous enseignent les crises du pass_.pdf
Processing file: Ce rapport au vitriol de la Cour des comptes sur le Grand Paris.pdf
File processed: Ce rapport au vitriol de la Cour des comptes sur le Grand Paris.pdf
Processing file: Cessons de creuser le trou de la dette publique.pdf
File processed: Cessons de creuser le trou de la dette publique.pdf
Processing file: Changement climatique _ Le Secr_taire g_n_ral de l'OCDE salue les appels lanc_s en faveur de nouvell.pdf
File processed: Changement climatique _ Le Secr_taire g_n_ral de l'OCDE salue les appels lanc_s en faveur de nouvell.pdf
Processing file: Christian Lindner, le grand argentier, justifie les volte-face de l'Allemagne.pdf
File processed: Christian Lindner, le grand argentier, justifie les volte-face de l'Allemagne.pdf
Processing file: CHRONIQUE Le baby krach.pdf
File pro

...

File processed: Insouciance ou incons_quence des dirigeants _.pdf
Processing file: Interventionnisme et _br_silianisation_ des prix du p_trole _ le menu _conomique de Lula.pdf
File processed: Interventionnisme et _br_silianisation_ des prix du p_trole _ le menu _conomique de Lula.pdf
Processing file: Investir est un imp_ratif_ mais peut devenir une opportunit_.pdf
File processed: Investir est un imp_ratif_ mais peut devenir une opportunit_.pdf
Processing file: In_galit_s _ la guerre des g_n_rations n'a pas lieu d'_tre.pdf
File processed: In_galit_s _ la guerre des g_n_rations n'a pas lieu d'_tre.pdf
Processing file: ITALIE _ La d_claration de politique g_n_rale de la nouvelle pr_sidente du conseil Madame Giorgia ME.pdf
File processed: ITALIE _ La d_claration de politique g_n_rale de la nouvelle pr_sidente du conseil Madame Giorgia ME.pdf
Processing file: Italie _ les familles politiques _ l'heure du bilan.pdf
File processed: Italie _ les familles politiques _ l'heure du bilan.pdf
Proce

File processed: L'immigration n'est pas la solution pour enrayer le d_clin d_mographique fran_ais.pdf
Processing file: L'Italie n'arr_te pas de r_viser sa croissance _ la hausse.pdf
File processed: L'Italie n'arr_te pas de r_viser sa croissance _ la hausse.pdf
Processing file: L'Occident assi_g_.pdf
File processed: L'Occident assi_g_.pdf
Processing file: L'OCDE alerte sur _la bombe _ retardement_ de la dette publique.pdf
File processed: L'OCDE alerte sur _la bombe _ retardement_ de la dette publique.pdf
Processing file: L'OCDE approuve les plans de rigueur fran_ais, allemand et britannique.pdf
File processed: L'OCDE approuve les plans de rigueur fran_ais, allemand et britannique.pdf
Processing file: L'OCDE presse les gouvernements d'agir _ maintenant _ pour r_duire leur dette.pdf
File processed: L'OCDE presse les gouvernements d'agir _ maintenant _ pour r_duire leur dette.pdf
Processing file: L'UE lance un d_bat difficile sur l'avenir de son corset budg_taire.pdf
File processed: L'UE l

File processed: La lettre du Figaro du 22 juin 2022.pdf
Processing file: La lettre du Figaro du 30 novembre 2020.pdf
File processed: La lettre du Figaro du 30 novembre 2020.pdf
Processing file: La lettre du Figaro du 5 septembre 2023.pdf
File processed: La lettre du Figaro du 5 septembre 2023.pdf
Processing file: La lettre du Figaro du 8 f_vrier 2022.pdf
File processed: La lettre du Figaro du 8 f_vrier 2022.pdf
Processing file: La le_on fran_aise du r_teau budg_taire allemand.pdf
File processed: La le_on fran_aise du r_teau budg_taire allemand.pdf
Processing file: La Maison-Blanche oblig_e de n_gocier pour _viter une catastrophe financi_re.pdf
File processed: La Maison-Blanche oblig_e de n_gocier pour _viter une catastrophe financi_re.pdf
Processing file: La mise au pas du monde de la finance est-elle possible_.pdf
File processed: La mise au pas du monde de la finance est-elle possible_.pdf
Processing file: La M_tropole pr_voit d'investir 570 millions en 2024.pdf
File processed: La M_t

File processed: Le gouvernement veut revoir les concessions autorouti_res.pdf
Processing file: Le gouvernement _plus pr_occup_ par la dette publique_ qu'__cologique_ (Lambert, LREM).pdf
File processed: Le gouvernement _plus pr_occup_ par la dette publique_ qu'__cologique_ (Lambert, LREM).pdf
Processing file: Le Grand d_bat national, une occasion de changer de voie.pdf
File processed: Le Grand d_bat national, une occasion de changer de voie.pdf
Processing file: Le Japon face aux d_fis de l'_re Reiwa.pdf
File processed: Le Japon face aux d_fis de l'_re Reiwa.pdf
Processing file: Le macronisme est-il soluble dans le n_olib_ralisme _.pdf
File processed: Le macronisme est-il soluble dans le n_olib_ralisme _.pdf
Processing file: Le manifeste des _Atterr_s_.pdf
File processed: Le manifeste des _Atterr_s_.pdf
Processing file: Le Medef met la libert_ au programme de la prochaine campagne pr_sidentielle.pdf
File processed: Le Medef met la libert_ au programme de la prochaine campagne pr_sidentie

File processed: Les finances de la France sont pass_es au tamis de la transition _cologique.pdf
Processing file: Les finances portugaises se portent _ merveille, Moody s rel_ve fortement sa note.pdf
File processed: Les finances portugaises se portent _ merveille, Moody s rel_ve fortement sa note.pdf
Processing file: Les finances publiques Jusqu'au second tour, _La Croix_ compare les programmes des deux finalistes s.pdf
File processed: Les finances publiques Jusqu'au second tour, _La Croix_ compare les programmes des deux finalistes s.pdf
Processing file: Les Fran_ais sont-ils accros _ la d_pense publique _ Le projet de budget 2024 doit _tre examin_ _ pa.pdf
File processed: Les Fran_ais sont-ils accros _ la d_pense publique _ Le projet de budget 2024 doit _tre examin_ _ pa.pdf
Processing file: Les gagnants et les perdants des _lections de mi-mandat aux Etats-Unis.pdf
File processed: Les gagnants et les perdants des _lections de mi-mandat aux Etats-Unis.pdf
Processing file: Les grands ou

In [2]:
# Load the Excel document
df_fr = pd.read_excel("C:\\Users\\eugen\\corpus_article_debt\\corpus_article_debt_FR.xlsx")

# Remove " words" from the "Length" column
df_fr['Length'] = df_fr['Length'].str.replace(" words", "", regex=False)

# Continue working with df as needed without saving the modified file

BadZipFile: File is not a zip file

In [21]:
# Convert the "Date" column to datetime objects
df_fr['Date'] = pd.to_datetime(df_fr['Date'], format='%Y%m%d')

# Format the dates in the "Date" column to "YYYY/MM/DD"
df_fr['Date'] = df_fr['Date'].dt.strftime('%Y/%m/%d')

NameError: name 'df_fr' is not defined

In [10]:
# Specify the file path
file_path = "C:\\Users\\eugen\\corpus_article_debt\\corpus_article_debt_FR.xlsx"

# Save the modified DataFrame to the same Excel file
df_fr.to_excel(file_path, index=False)